In [1]:
import os
import psycopg2
from psycopg2.extras import RealDictCursor
from sqlalchemy import create_engine
from psycopg2.extras import execute_values
import pandas as pd
import FinanceDataReader as fdr

In [2]:
# host="34.64.149.167"
host="121.134.7.122"
# host = "host.docker.internal"
port="5432"
dbname="finance_db"
user="postgres"
password="ekgkaehddudABC123!"
# password="lee1367"

# DB 연결
conn = psycopg2.connect(
    host=host,
    port=port,
    dbname=dbname,
    user=user,
    password=password
)
cur = conn.cursor()

In [3]:
cur.execute("""
SELECT table_name 
FROM information_schema.tables 
WHERE table_schema = 'public';
""")
tables = cur.fetchall()
for t in tables:
    print(t[0])

lotto_numbers
test_table
users
email_verifications


In [4]:
cur.execute("""
SELECT table_name 
FROM information_schema.tables 
WHERE table_schema = 'finance';
""")
tables = cur.fetchall()
for t in tables:
    print(t[0])

stock
financial_account
financial_statement_raw


In [ ]:
# 1) 기존 테이블 삭제 후 새로 생성
drop_query = "DROP TABLE IF EXISTS test_table;"
create_query = """
CREATE TABLE test_table (
    code   varchar not null,
    isu_cd varchar not null,
    name   varchar not null,
    market varchar not null
)
"""
cur.execute(drop_query)
cur.execute(create_query)
conn.commit()


In [ ]:
# 2) 단일 INSERT 예시
insert_query = """
INSERT INTO test_table (code, isu_cd, name, market) VALUES (%s, %s, %s, %s)
"""
params = (1111, 'aa', 'bb', 'cc')
cur.execute(insert_query, params)
conn.commit()


In [ ]:
# 3) 판다스로 bulk insert
df_krx = fdr.StockListing('KRX')
df = df_krx[['Code', 'ISU_CD', 'Name', 'Market']]
df.columns = df.columns.str.lower()

# DataFrame → list of tuples
values = [tuple(x) for x in df.to_numpy()]
columns = ','.join(df.columns)

# bulk insert 쿼리
insert_query = f"INSERT INTO test_table ({columns}) VALUES %s"

# bulk insert 실행
execute_values(cur, insert_query, values)
conn.commit()

print("✅ 데이터 삽입 완료")


In [4]:
# 2) 데이터 조회
df_result = pd.read_sql("SELECT * FROM test_table LIMIT 10;", conn)
print(df_result)

   year quarter stock_code fs_code             값
0  2015      Q4     000020   당기순이익  1.983311e+10
1  2016      Q1     000020   당기순이익  9.081832e+09
2  2016      Q2     000020   당기순이익  1.601060e+10
3  2016      Q3     000020   당기순이익  1.978969e+10
4  2016      Q4     000020   당기순이익  2.548219e+10
5  2017      Q1     000020   당기순이익  2.614052e+09
6  2017      Q2     000020   당기순이익  5.451184e+09
7  2017      Q3     000020   당기순이익  5.731737e+09
8  2017      Q4     000020   당기순이익 -1.651997e+10
9  2018      Q1     000020   당기순이익  1.354203e+10


/tmp/ipykernel_31262/2914732984.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_result = pd.read_sql("SELECT * FROM test_table LIMIT 10;", conn)


In [5]:
temp = pd.read_sql("SELECT * FROM users LIMIT 20;", conn)
temp

/tmp/ipykernel_48897/377821211.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  temp = pd.read_sql("SELECT * FROM users LIMIT 20;", conn)


,username,email,password_hash,is_active,is_verified,email_verified_at,last_login_at,nickname,full_name,birth_date,...,subscription_started_at,subscription_expires_at,subscription_auto_renew,login_count,failed_login_attempts,locked_until,deleted_at,id,created_at,updated_at
0,daham1224,daham1224@naver.com,$pbkdf2-sha256$29000$9H6vNcb4v9c6Z6y1dq4VAg$tU...,True,False,None,2025-10-26 13:41:51.890166,팬더고기,이다함,1989-12-24,...,None,None,False,2,0,None,None,1,2025-10-26 12:55:11.993475+00:00,2025-10-26 13:41:51.994919+00:00


In [3]:
temp2 = pd.read_sql("SELECT * FROM email_verifications LIMIT 20;", conn)
temp2

/tmp/ipykernel_1685/3608346569.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  temp2 = pd.read_sql("SELECT * FROM email_verifications LIMIT 20;", conn)


,email,code,expires_at,is_used,id,created_at,updated_at
0,test@example.com,784936,2025-10-26 12:54:34.104310,false,3,2025-10-26 12:44:34.223339+00:00,None
1,daham1224@naver.com,263452,2025-10-26 13:00:20.763771,false,4,2025-10-26 12:50:20.885066+00:00,None


In [9]:
temp.columns

Index(['username', 'email', 'password_hash', 'is_active', 'is_verified',
       'email_verified_at', 'last_login_at', 'birth_date', 'gender', 'phone',
       'full_name', 'nickname', 'profile_image_url', 'marketing_agreed',
       'marketing_agreed_at', 'terms_agreed', 'terms_agreed_at',
       'privacy_agreed', 'privacy_agreed_at', 'role', 'is_premium',
       'premium_expires_at', 'subscription_type', 'subscription_started_at',
       'subscription_expires_at', 'subscription_auto_renew', 'login_count',
       'failed_login_attempts', 'locked_until', 'deleted_at', 'id',
       'created_at', 'updated_at'],
      dtype='object')

In [ ]:
cur.close()
conn.close()

In [7]:
temp = pd.read_sql("SELECT * FROM test_table;", conn)

/tmp/ipykernel_5087/2212165809.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  temp = pd.read_sql("SELECT * FROM test_table;", conn)


In [9]:
temp

,year,quarter,stock_code,fs_code,값
0,2015,Q4,000020,당기순이익,1.983311e+10
1,2016,Q1,000020,당기순이익,9.081832e+09
2,2016,Q2,000020,당기순이익,1.601060e+10
3,2016,Q3,000020,당기순이익,1.978969e+10
4,2016,Q4,000020,당기순이익,2.548219e+10
...,...,...,...,...,...
545917,2024,Q2,950190,EPS,6.229098e-01
545918,2024,Q3,950190,EPS,9.103204e-01
545919,2024,Q4,950190,EPS,9.384307e-01
545920,2025,Q1,950190,EPS,2.542763e-01
